# Preliminary Info
Summarizes: 
https://link.springer.com/article/10.1140/epjc/s10052-021-09538-2
https://twiki.cern.ch/twiki/pub/CMSPublic/WorkBookExercisesHATS_Pileup_2013/Pileup_Overview_HATS13.pdf

Several main processes contribute to the total $pp$ cross section; non-diffractive ($pp\rightarrow X$), elastic ($pp\rightarrow pp$), and diffractive (single dissociation $pp\rightarrow Xp$ or $pp\rightarrow pY$, double dissociation $pp\rightarrow XY$, or central dissociation $pp\rightarrow pXp$). The inelastic cross section is everything except elastic scattering; pileup is produced by inelastic processes (minbias events). 

In MC, the instantaneous luminosity for an event is sampled to give a mean number of interactions in each beam crossing. This number of interactions is used as a Poisson mean. The observed pileup takes these Poisson fluctuations into account. 

The bunch-bunch collision luminosity is measured by the van der Meer metod. The instantaneous luminosity for a bunch pair collision is $\mathcal{L}_b=\frac{f n_1 n_2}{A_\mathrm{eff}}$ where $A_\mathrm{eff}$ counts the effective overlap area between the bunches. Given density functions for the bunches, $A_\mathrm{eff}$ is the integral,

$$A_\mathrm{eff}^{-1} = \int\int\rho_1(x, y)\rho_2(x,y)dxdy$$

Here $\rho_1(x,y)$ and $\rho_2(x,y)$ are normalized density distributions in the transverse plane. Assuming Gaussian-distributed bunches with equal widths and heights, this integral factorizes:

$$\mathcal{L}_b = \frac{fn_1n_2}{4\pi\sigma_x\sigma_y}$$

Assuming Gaussian beams displaced by some relative $w$ or $h$ in the $x$ or $y$ direction, the beams have new effective widths $\Sigma_i = \sqrt{2}\sigma_i$, giving

$$\mathcal{L}_b = \frac{fn_1n_2}{2\pi \Sigma_x \Sigma_y}$$

To recover the LHC instantaneous luminosity, we simply mulitply by $n_\mathrm{bunches}$, which accounts for the fact that for each revolution, every bunch will be crossed. 

$$\mathcal{L} = n_\mathrm{bunches}\times\frac{fn_1n_2}{2\pi \Sigma_x \Sigma_y}$$

The inelastic rate is $R_\mathrm{inel}=\mathcal{L}\sigma_\mathrm{inel}$.

To perform a measurement, the luminosity is extracted from the visible inelastic collision rate. Given an agreed-upon $\sigma_\mathrm{inel}=69.2mb$, one can extract the average pileup $\mu$. 

Summarizes: https://twiki.cern.ch/twiki/bin/viewauth/CMS/PileupJSONFileforData

Given an instantaneous luminosity $\mathcal{L}^i$ in a single bunch, the pileup in the bunch crossing is $\mu = (\mathcal{L}^i/f)\sigma_\mathrm{inel}$, where $f=11246$ Hz is the LHC orbit frequency. This gives the per-collision luminosity, $\mathcal{L}=\mathcal{L}^i/f$. Lumi-sections are defined to be $\approx 23.3$ seconds, and often people quote $\mu=(\mathcal{L}\times 23.3s)\sigma_\mathrm{inel}$. This is a pileup *average*, so we take it to be the mean of true Poissson pileup distribution. 

This lumi-based estimation can be compared with the number of PVs reconstructed in the event, which is subject to a $70\%$ reconstruction efficiency. 

The pileupCalc tool extracts the pileup distributions relevant for an analysis. For Run 2, the minimum bias cross section is taken be $69200\mu b$ with an uncertainty of $4.6\%$. 

The pileup reweighting histograms are provided here:

2018: /afs/cern.ch/cms/CAF/CMSCOMM/COMM_DQM/certification/Collisions18/13TeV/PileUp/UltraLegacy/
2017: /afs/cern.ch/cms/CAF/CMSCOMM/COMM_DQM/certification/Collisions17/13TeV/PileUp/UltraLegacy/
2016: /afs/cern.ch/cms/CAF/CMSCOMM/COMM_DQM/certification/Collisions16/13TeV/PileUp/UltraLegacy/


Pileup events in MC are generated from "true" pileup values, which are Poisson means (`numTrueInteractions`). The provided pileup histograms contain one entry per lumisection, the average pileup for the lumisection, as calculated by $\mu=(\mathcal{L}^i\times 23.3s)\sigma_\mathrm{inel}$. This is exactly the mean expected from MC. 


## Pileup Reweighting
Summarizes: https://twiki.cern.ch/twiki/pub/CMSPublic/WorkBookExercisesHATS_Pileup_2013/Pileup_Overview_HATS13.pdf

MC events are pileup reweighted to account for differences between data and MC. The ratio of (target/input) is used to derive weights. The data pileup distribution is calculated from the bunch-by bunch luminosities. These measurements give the mean number of interactions per bunch crossing. The actual observation is a Poisson distribution for each bunch crossing. This is very important. 

In [ ]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict
import os
import sys
sys.path.append('../')
import hist
import mplhep as hep
import seaborn as sns
import numpy as np
from coffea import util
import uproot
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
#from azh_analysis.utils.plotting import plot_variable
import warnings
warnings.filterwarnings('ignore')
hep.style.use(["CMS", "fira", "firamath"])

In [ ]:
from azh_analysis.utils.pileup import open_pileup_file
from hist.intervals import ratio_uncertainty
samples = ['DY1JetsToLLM-50_2018', 'DY2JetsToLLM-50_2018', 'DY3JetsToLLM-50_2018', 'DY4JetsToLLM-50_2018', 'DYJetsToLLM-50_2018', 'DYJetsToLLM-50_ext1_2018', 'GluGluToContinToZZTo2e2mu_2018', 'GluGluToContinToZZTo2e2tau_2018', 'GluGluToContinToZZTo2mu2tau_2018', 'GluGluToContinToZZTo4e_2018', 'GluGluToContinToZZTo4mu_2018', 'GluGluToContinToZZTo4tau_2018', 'GluGluZHHToWW_2018', 'HWminusJHToWW_2018', 'HWplusJHToWWTo2L2Nu_2018', 'HZJHToWW_2018', 'TTTo2L2Nu_2018', 'TTToHadronic_2018', 'TTToSemiLeptonic_2018', 'TTWJetsToLNu_2018', 'VBFHToTauTauM125_2018', 'VBFHToWWTo2L2NuM-125_2018', 'WWW4F_2018', 'WWW4F_ext1_2018', 'WWZ4F_2018', 'WZTo2Q2Lmllmin4p0_2018', 'WZTo3LNu_2018', 'WZZTuneCP5_ext1_2018', 'WZZ_2018', 'WminusHToTauTauM125_2018', 'WplusHToTauTauM125_2018', 'ZHToTauTauM125_ext1_2018', 'ZZTo2Q2Lmllmin4p0_2018', 'ZZTo4L_2018', 'ZZZTuneCP5_ext1_2018', 'ZZZ_2018', 'ttHToTauTauM125_2018', 'ttZJets_2018']

# load up MC
mc_pu = util.load("../corrections/pileup/UL_2018/MC_UL_2018_PU.coffea")["pileup_mc"]
mc_pu = sum([mc_pu[d, :].to_hist() for d in samples])
mc_pu, mc_bins = mc_pu[::sum,:].values(), mc_pu[::sum,:].axes['pileup'].centers
mc_pu = (mc_pu / np.sum(mc_pu))[:-1]
mc_bins = mc_bins[:-1]
print(mc_bins)

# draw data
indir = '../corrections/pileup/UL_2018'
data_pu, bins = open_pileup_file(year="2018", indir="../corrections/pileup")
data_bins = (bins[1:] + bins[:-1])/2.
data_pu = data_pu / np.sum(data_pu)

# build figure
fig, (ax, rax) = plt.subplots(nrows=2, ncols=1, figsize=(10, 12), dpi=80,
                              gridspec_kw={"height_ratios": (4, 1)}, sharex=True)
fig.subplots_adjust(hspace=.07)
ax.set_prop_cycle(cycler(color=['#EE9B00','#005F73']))

# avoid off-by-one error
ax.plot(mc_bins+1, mc_pu, marker='.', label="MC")
ax.plot(data_bins, data_pu, marker='.', 
        label="Data (69.2mb)")

bins = mc_bins
weights = data_pu/mc_pu
rax.errorbar(
    x=mc_bins,
    y=weights,
    yerr=ratio_uncertainty(data_pu, mc_pu, "poisson-ratio"),
    color="k",
    linestyle="none",
    marker="o",
    elinewidth=1.25,
    capsize=2,
)

rax.set_xlabel(r"$\langle \mu \rangle$")
ax.set_ylabel('Density')
rax.set_ylabel("Pileup Weight")
ax.legend(loc="best", prop={'size': 16}, frameon=True)
rax.set_ylim([0, 2])
hep.cms.label('Preliminary', data=True, lumi=59.7, year=2018, ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
from azh_analysis.utils.corrections import get_pileup_weights
pu_evaluator = get_pileup_weights("../corrections/pileup/", year="2017")
pu_evaluator

In [ ]:
from azh_analysis.utils.pileup import make_pileup_weights_file
make_pileup_weights_file("../corrections/pileup", "2016preVFP")

In [ ]:
from azh_analysis.utils.corrections import get_pileup_weights
d = get_pileup_weights("../corrections/pileup/", "2016preVFP")

In [ ]:
d["nom"](30)

In [ ]:
from coffea import analysis_tools
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
base = '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2018/DY4JetsToLLM-50'
file = os.path.join(base, 'all_DY4JetsToLLM-50_file001_part_1of3_Electrons.root')
events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
weights = analysis_tools.Weights(len(events), storeIndividual=True)
weights.add("sample", np.ones(len(events)))
weights.add("sample2", 0.5*np.ones(len(events)))
weights.add(
    "pileup", 
    weight=d["nom"](events.Pileup.nTrueInt),
    weightUp=d["up"](events.Pileup.nTrueInt),
    weightDown=d["down"](events.Pileup.nTrueInt),
)
weights.add(
    "l1prefire", 
    weight=0.5*events.L1PreFiringWeight.Nom,
    weightUp=events.L1PreFiringWeight.Up,
    weightDown=events.L1PreFiringWeight.Dn,
)

In [ ]:
print(weights.partial_weight(include=["l1prefire"]))
print(weights.partial_weight(include=["pileup"]))
print(weights.partial_weight(include=["sample2"]))

In [ ]:
print(weights.weight())
print(weights.partial_weight(include=['sample2', "pileup", "l1prefire"]))

In [ ]:
mask = np.random.rand(len(events)) > 0.5
weights[mask]

In [ ]:
weights.weight(modifier="pileupDown")

In [ ]:
events.L1PreFiringWeight.Up[events.L1PreFiringWeight.Nom<1]